<a href="https://colab.research.google.com/github/WahyuKhairi06/DeepLearning-WahyuKhairi-2311531009/blob/main/Praktikum%2010/Deep_Belief_Network_(DBN)_dan_Capsule_Network_(CapsNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A. Implementasi Deep Belief Network (DBN)**

In [6]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
import numpy as np


**1. Load dataset MNIST**

In [7]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.astype('float32')
X = MinMaxScaler().fit_transform(X)
y = y.astype('int')


**2. Definisikan RBM dan Logistic Regression**


In [8]:
rbm = BernoulliRBM(n_components=256, learning_rate=0.06, batch_size=10, n_iter=10, verbose=1)
logistic = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')


**3. DBN sebagai pipeline RBM + Logistic Regression**

In [9]:
dbn = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
dbn.fit(X[:60000], y[:60000])



[BernoulliRBM] Iteration 1, pseudo-likelihood = -87.11, time = 26.57s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -83.41, time = 28.13s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -82.35, time = 28.38s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -81.32, time = 28.36s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -82.21, time = 34.64s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -82.41, time = 28.47s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -80.34, time = 28.75s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -81.41, time = 28.29s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -82.98, time = 28.83s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -81.10, time = 28.24s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('rbm', BernoulliRBM(learning_rate=0.06, verbose=1)),
                ('logistic',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

**4. Evaluasi**

In [10]:

score = dbn.score(X[60000:], y[60000:])
print(f"Akurasi DBN pada data uji: {score * 100:.2f}%")


Akurasi DBN pada data uji: 96.46%


# **B. Implementasi Capsule Network (CapsNet)**

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# **1. Load dataset**

In [12]:

transform = transforms.Compose([transforms.ToTensor()])
train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(datasets.MNIST('./data', train=False, transform=transform), batch_size=64, shuffle=False)


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.47MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.61MB/s]


**2. Definisi CapsNet sederhana**

In [13]:

class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, num_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_route_nodes = num_route_nodes
        self.num_capsules = num_capsules
        self.num_iterations = num_iterations
        self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))

    def squash(self, tensor, dim=-1):
        norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

    def forward(self, x):
        x = x[None, :, :, None, :] @ self.route_weights[None, :, :, :, :]
        x = x.squeeze(3)
        b = torch.zeros_like(x[:, :, :, 0])
        for i in range(self.num_iterations):
            c = F.softmax(b, dim=2)
            s = (c.unsqueeze(4) * x).sum(dim=2)
            v = self.squash(s)
            if i < self.num_iterations - 1:
                b = b + (x * v[:, :, None, :]).sum(dim=-1)
        return v

class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 256, kernel_size=9, stride=1)
        self.primary_caps = nn.Conv2d(256, 8 * 32, kernel_size=9, stride=2, groups=8)
        self.digit_caps = CapsuleLayer(num_capsules=10, num_route_nodes=32*6*6, in_channels=8, out_channels=16)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.primary_caps(x)
        x = x.view(x.size(0), 32*6*6, 8)
        x = self.digit_caps(x)
        return (x ** 2).sum(dim=-1) ** 0.5

model = CapsNet()
print(model)


CapsNet(
  (conv1): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  (primary_caps): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2), groups=8)
  (digit_caps): CapsuleLayer()
)
